In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/tp2datostrain/train.csv
/kaggle/input/tp2datostrain/test.csv
/kaggle/input/pingsound/PING - Sound effect.mp3


In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from category_encoders import CountEncoder
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from IPython.display import Audio
import matplotlib.pyplot as plt

In [3]:
# Cargo los archivos
sound_file = '/kaggle/input/pingsound/PING - Sound effect.mp3'
test = pd.read_csv("../input/tp2datostrain/test.csv")
display(test.shape)
train = pd.read_csv("../input/tp2datostrain/train.csv")
display(train.shape)

(60000, 22)

(240000, 23)

In [35]:
def ping():
    display(Audio(sound_file, autoplay=True))

In [5]:
dfCheto = pd.concat([train,test])
df = dfCheto.copy()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [6]:
ciudades = list(dfCheto["ciudad"].unique())
tipos_de_propiedad = list(dfCheto["tipodepropiedad"].unique())
provincia = list(dfCheto["provincia"].unique())
categorias  = [provincia, tipos_de_propiedad, ciudades]

### Preproceso el DF
#### Relleno nulos, etc.

In [7]:
#Swapeo los metros
df["swapear"] = df["metrostotales"] < df["metroscubiertos"]
df_metros_arreglados = df.copy()
aux = df_metros_arreglados.loc[df_metros_arreglados["swapear"], "metrostotales"]
if (not aux.empty):
    df_metros_arreglados.loc[df_metros_arreglados["swapear"],"metrostotales"] = \
    df_metros_arreglados.loc[df_metros_arreglados["swapear"], "metroscubiertos"]
    df_metros_arreglados.loc[df_metros_arreglados["swapear"], "metroscubiertos"] = aux
df_metros_arreglados.drop(columns="swapear", inplace=True)
(df_metros_arreglados["metrostotales"] < df_metros_arreglados["metroscubiertos"]).value_counts()
df = df_metros_arreglados

In [8]:
#Relleno las ciu|dades nuelas
df['ciudad'] = SimpleImputer(strategy='constant', fill_value='dummy').fit_transform(df[['ciudad']])
#df['ciudad'] = CountEncoder().fit_transform(df['ciudad'])

In [9]:
#RELLENO LOS IDS CON LO QUE MAS APARECE EN LA CIUDAD
def obtenerMasFrecuente(x):
    x = pd.Series.mode(x)
    s = x.loc[x>0]
    if len(s.index)<=0:
        return np.nan
    return s.value_counts().index[0]
print("Nulos idzona: ", df["idzona"].isna().value_counts())
print("Nulos coidad: ", df["ciudad"].isna().value_counts())

aux = df[["ciudad", "idzona"]].copy()
idzonaMasFrecuentePorCiudad = aux.groupby('ciudad')["idzona"].transform(lambda x: obtenerMasFrecuente(x))
idzonaMasFrecuentePorCiudad

Nulos idzona:  False    264200
True      35800
Name: idzona, dtype: int64
Nulos coidad:  False    300000
Name: ciudad, dtype: int64


0        50003995.0
1           24481.0
2           48638.0
3           53666.0
4           47735.0
            ...    
59995       25039.0
59996       57611.0
59997       72223.0
59998       84312.0
59999       47735.0
Name: idzona, Length: 300000, dtype: float64

In [10]:
df.idzona.fillna(idzonaMasFrecuentePorCiudad, inplace = True)
display("Nulos idzona: ", df["idzona"].isna().value_counts())
#DROPEO LAS COLUMNAS QUE QUEDARON SIN NADA si estoy entrenando o relleno con la mas comun si estoy testeando
df['idzona'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['idzona']])
display("Nulos idzona: ", df["idzona"].isnull().value_counts())
df.shape

'Nulos idzona: '

False    299431
True        569
Name: idzona, dtype: int64

'Nulos idzona: '

False    300000
Name: idzona, dtype: int64

(300000, 23)

In [11]:
def rellenar_con_X_por_grupo(df, grupo, rellenar, func):
    valoresAgrupados = df.groupby(grupo)[rellenar].transform(func)
    df[rellenar].fillna(valoresAgrupados, inplace=True)

In [12]:
#Reemplazo las habitaciones por la mediana de la zona
print("Nulos habitaciones: ", df["habitaciones"].isnull().value_counts())
rellenar_con_X_por_grupo(df, "idzona", "habitaciones", lambda x: x.median())
print("Nulos habitaciones: ", df["habitaciones"].isnull().value_counts())
#Relleno el resto con las mas frecuentes
df['habitaciones'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['habitaciones']])
#Para la antiguedad relleno con el promedio de la zona
print("Nulos antiguedad: ", df["antiguedad"].isna().value_counts())
medianaAntiguedadXZona = df.groupby("idzona")["antiguedad"].transform(lambda x: x.median())
df["antiguedad"].fillna(medianaAntiguedadXZona, inplace = True)
print("Nulos antiguedad: ", df["antiguedad"].isna().value_counts())
#Reemplazo los banos por 1 que es lo minimo posible
df.banos.fillna(1, inplace=True)
#Relleno garages con 0 porque es lo mas probable
df.garages.fillna(0, inplace = True)
print("Nulos metrostotales: ", df["metrostotales"].isna().value_counts())
print("Nulos metroscubiertos: ", df["metroscubiertos"].isna().value_counts())
#COMO QUEDAN NULOS LOS RELLENO CON la mediana (porque la mediana es un valor, y es menos sensible a valores extremos) por ZONA / CIUDAD
promedioMetrosCubiertosXZona = df.groupby("idzona")["metroscubiertos"].transform(lambda x: x.median())
df["metroscubiertos"].fillna(promedioMetrosCubiertosXZona, inplace = True)
print("Nulos metroscubiertos: ", df["metroscubiertos"].isna().value_counts())
promedioMetrosTotalesXZona = df.groupby("idzona")["metrostotales"].transform(lambda x: x.median())
df["metrostotales"].fillna(promedioMetrosCubiertosXZona, inplace = True)
print("Nulos metrostotales: ", df["metrostotales"].isna().value_counts())

Nulos habitaciones:  False    271901
True      28099
Name: habitaciones, dtype: int64
Nulos habitaciones:  False    298667
True       1333
Name: habitaciones, dtype: int64
Nulos antiguedad:  False    245731
True      54269
Name: antiguedad, dtype: int64
Nulos antiguedad:  False    298314
True       1686
Name: antiguedad, dtype: int64
Nulos metrostotales:  False    235878
True      64122
Name: metrostotales, dtype: int64
Nulos metroscubiertos:  False    278301
True      21699
Name: metroscubiertos, dtype: int64
Nulos metroscubiertos:  False    299262
True        738
Name: metroscubiertos, dtype: int64
Nulos metrostotales:  False    300000
Name: metrostotales, dtype: int64


In [ ]:
aux = dfCheto[['lat', 'ciudad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('ciudad').agg({'lat':'mean'})
lats = aux.T.to_dict('records').copy()
lats = lats[0]
df.lat.fillna(df.ciudad.map(lats), inplace = True)
df['lat'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['lat']])
aux = dfCheto[['lng', 'ciudad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('ciudad').agg({'lng':'mean'})
lngs = aux.T.to_dict('records').copy()
lngs = lngs[0]
df.lng.fillna(df.ciudad.map(lngs), inplace = True)
df['lng'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['lng']])

### Agrego Features Texto

In [13]:
df["titulo_descripcion"] = df["titulo"].astype(str) + " " + df["descripcion"].astype(str)
#Como tiene algunos nulos los relleno con cadenas vaicas
df["titulo_descripcion"].fillna("",inplace = True)
#REMUEVO LA PUNTUACION
import string
print(string.punctuation)
tabla = {}
for c in string.punctuation + "¿¡":
    tabla[ord(c)] = None
def sacar_puntuacion(s):
    return s.translate(tabla)
df["titulo_descripcion"] = df["titulo_descripcion"].apply(sacar_puntuacion)
#Agrego los primeros features
df["nro_palabras"] = df["titulo_descripcion"].apply(lambda x: len(x.split(" ")))
df["nro_caracteres"] = df["titulo_descripcion"].apply(lambda x: len(x))
df["long_prom_palabra"] = df["nro_caracteres"] / df["nro_palabras"]

def llenarDiccionario(diccionario,descripcion):
    for palabra in descripcion.split():
        diccionario[palabra] = diccionario.get(palabra,0) + 1
        
def generarDiccionario(df2):
    dicc = {}
    for descripcion in df2['titulo_descripcion']:
        if isinstance(descripcion,str): llenarDiccionario(dicc,descripcion)
    return dicc
contador_palabras = generarDiccionario(df)

#Elimino las stopwords = palabras vacias que no suman nada
from nltk.corpus import stopwords
palabras_vacias = stopwords.words('spanish')

for palabra  in palabras_vacias:
    if palabra in contador_palabras:
        del contador_palabras[palabra]
ordenados = sorted(contador_palabras.items(),key = lambda x: -x[1])
top_50 = dict(ordenados[:50])

#Nuevos features
def contar_palabras_en_top(texto):
    palabras = texto.split()
    cont = 0
    for palabra in palabras:
        if palabra in top_50:
            cont += 1
    return cont
df["nro_palabras_mas_comunes"] = df["titulo_descripcion"].apply(contar_palabras_en_top)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


### Agrego feature de cantidad de publicaciones

In [14]:
df['publicacionesPorCiudad'] = CountEncoder().fit_transform(df['ciudad'])
df['publicacionesPorProvincia'] = CountEncoder().fit_transform(df['provincia'])
df['publicacionesPorTipo'] = CountEncoder().fit_transform(df['tipodepropiedad'])
df['publicacionesPorZona'] = CountEncoder().fit_transform(df['idzona'])
df['publicacionesPorBanos'] = CountEncoder().fit_transform(df['banos'])
df['publicacionesPorHabitaciones'] = CountEncoder().fit_transform(df['habitaciones'])
df['publicacionesPorAntiguedad'] = CountEncoder().fit_transform(df['antiguedad'])

In [15]:
df["fecha"] = pd.to_datetime(df["fecha"])
df.set_index('id', inplace = True)
columna_aux = df.columns[1]
df["nro_publicaciones_dia"] = df.groupby(pd.Grouper(key='fecha', freq='D'))[columna_aux].transform(lambda x: len(x.index))
df["nro_publicaciones_semana"] = df.groupby(pd.Grouper(key='fecha', freq='W'))[columna_aux].transform(lambda x: len(x.index))
df["nro_publicaciones_5_dias_habiles"] = df.groupby(pd.Grouper(key='fecha', freq='B'))[columna_aux].transform(lambda x: len(x.index))
df["nro_publicaciones_mes"] = df.groupby(pd.Grouper(key='fecha', freq='M'))[columna_aux].transform(lambda x: len(x.index))

In [16]:
df['fecha'] = pd.to_datetime(df['fecha'])
df['dia'] = df.fecha.dt.day
df['mes'] = df.fecha.dt.month
df['anio'] = df.fecha.dt.year

### Agrego features de características por grupo

In [41]:
df['promedioAntiguedadZona'] = df.groupby('idzona').agg({'antiguedad':'mean'})
df['promedioAntiguedadCiudad'] = df.groupby('ciudad').agg({'antiguedad':'mean'})
df['promedioHabitacionesZona'] = df.groupby('idzona').agg({'habitaciones':'mean'})
df['promedioHabitacionesCiudad'] = df.groupby('ciudad').agg({'habitaciones':'mean'})


## Creo los pipelines del preprocesador

In [42]:
features = [#cOLUMNAS
            'idzona',
            'lat', 'lng',
            "metrostotales", "metroscubiertos", 
            'garages', 'banos',
            'gimnasio','centroscomercialescercanos','usosmultiples','piscina','escuelascercanas',
            'antiguedad',
            'habitaciones',
            #Numero de publicaciones
            'nro_publicaciones_dia',
            'nro_publicaciones_semana',
            'nro_publicaciones_5_dias_habiles',
            'nro_publicaciones_mes',
            'publicacionesPorCiudad', 'publicacionesPorProvincia', 'publicacionesPorTipo',
            'publicacionesPorBanos', 'publicacionesPorHabitaciones', 'publicacionesPorAntiguedad',
            #texto
            "nro_palabras", "nro_caracteres", "long_prom_palabra", "nro_palabras_mas_comunes",
            #categoricas
            'provincia','tipodepropiedad', 'ciudad',
            #fecha
            'mes', 'dia', 'anio',
            #Promedios por grupo
    
            'promedioAntiguedadZona', 'promedioAntiguedadCiudad',
            'promedioHabitacionesZona', 'promedioHabitacionesCiudad',
           ]

In [43]:
transformador_categorias = make_pipeline(SimpleImputer(strategy='most_frequent'),
                                         OneHotEncoder(handle_unknown='ignore', categories = categorias))
transformador_numeros = make_pipeline(SimpleImputer())

In [44]:
def obtenerFeaturesCategoricos(features):
    l = []
    for feature in features:
        if not np.issubdtype(df[feature].dtype, np.number):
            l.append(feature)
    return l

num_features = list(df[features].select_dtypes(include=[np.number]).columns)

cat_features = list(obtenerFeaturesCategoricos(features))
print ("Features numericos:", num_features, '\nFeatures categoricos: ', cat_features)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformador_numeros, num_features),
        ('cat', transformador_categorias, cat_features)
    ])

Features numericos: ['idzona', 'metrostotales', 'metroscubiertos', 'garages', 'banos', 'gimnasio', 'centroscomercialescercanos', 'usosmultiples', 'piscina', 'escuelascercanas', 'antiguedad', 'habitaciones', 'nro_publicaciones_dia', 'nro_publicaciones_semana', 'nro_publicaciones_5_dias_habiles', 'nro_publicaciones_mes', 'publicacionesPorCiudad', 'publicacionesPorProvincia', 'publicacionesPorTipo', 'publicacionesPorBanos', 'publicacionesPorHabitaciones', 'publicacionesPorAntiguedad', 'nro_palabras', 'nro_caracteres', 'long_prom_palabra', 'nro_palabras_mas_comunes', 'mes', 'dia', 'anio', 'promedioAntiguedadZona', 'promedioAntiguedadCiudad', 'promedioHabitacionesZona', 'promedioHabitacionesCiudad'] 
Features categoricos:  ['provincia', 'tipodepropiedad', 'ciudad']


In [45]:
model = xgb.XGBRegressor(learning_rate= 0.1,n_estimators= 2000,scale_pos_weight= 2,
                         max_depth= 5,min_child_weight= 5,gamma= 0.0,
                         colsample_bytree= 0.75,
                         subsample= 0.7,colsample_bylevel= 0.65,
                         eval_metric = "mae",
                         objective='reg:squarederror',
                          tree_method='gpu_hist', gpu_id=0)

In [46]:
display(test.shape)
display(train.shape)
entrenamiento = df.head(240000).copy()
prueba = df.tail(60000).copy()
prueba.drop(columns=['precio'], inplace=True)

(60000, 22)

(240000, 23)

In [47]:
X = entrenamiento[features].copy()
y = entrenamiento.precio.copy()

In [29]:
pipeline = make_pipeline(preprocessor, model, verbose = True)

In [30]:
train_x, test_x, train_y, test_y = train_test_split(X, y,train_size=0.8, test_size=0.2)
pipeline.fit(train_x, train_y)
preds = pipeline.predict(test_x)
from sklearn.metrics import mean_absolute_error
mean_absolute_error(preds, test_y)

[Pipeline] . (step 1 of 2) Processing columntransformer, total=  42.2s


/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[Pipeline] ...... (step 2 of 2) Processing xgbregressor, total=  17.0s


589638.5259411875

In [ ]:
from sklearn.model_selection import cross_val_score
SCORE = "mean_absolute_error"
def cross_validation(pipeline, X, y, nro_folds):
    score = -1 * cross_val_score(pipeline, X, y,
                              cv=nro_folds,
                              scoring='neg_mean_absolute_error')
    print("Puntajes: ", end="")
    for p in score:
        print(p, ", ", end="")
    
    print("\nPromedio: ", np.mean(score))
    
cross_validation(pipeline, X, y, 3)
ping()

[Pipeline] . (step 1 of 2) Processing columntransformer, total=  25.6s


/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[Pipeline] ...... (step 2 of 2) Processing xgbregressor, total=  15.7s
[Pipeline] . (step 1 of 2) Processing columntransformer, total=  28.6s


/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[Pipeline] ...... (step 2 of 2) Processing xgbregressor, total=  16.0s


In [34]:
Xt, Tv, yt, Tyv = train_test_split(X, y,train_size=0.8, test_size=0.2, random_state = 0)
Tv2 = preprocessor.fit_transform(Tv)
Xt2 = preprocessor.fit_transform(Xt)
pipeline.fit(X,y, xgbregressor__eval_set=[(Xt2, yt.to_numpy()), (Tv2, Tyv.to_numpy())])
preds = pipeline.predict(Tv)
predictions = [round(value) for value in preds]
# evaluate predictions
#accuracy = accuracy_score(Ty, preds)
#print("Accuracy: %.2f%%" % (accuracy * 100.0))

KeyboardInterrupt: 

In [ ]:
# retrieve performance metrics
results = model.evals_result()
epochs = len(results['validation_0']['mae'])
x_axis = range(0, epochs)
# plot log loss
fig, ax = plt.subplots()
ax.plot(x_axis, results['validation_0']['mae'], label='Train')
ax.plot(x_axis, results['validation_1']['mae'], label='Test')
ax.legend()
plt.ylabel('mae')
plt.title('XGBoost mae')

In [ ]:
# import the modules we'll need
from IPython.display import HTML
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "resultados.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

predicciones_submit = pipeline.predict(prueba)
res = pd.DataFrame(predicciones_submit, index=test.id, columns=['precio'])
res.reset_index(inplace=True)
res.columns = ["id", "target"]
display(res.head())
# create a link to download the dataframe
display(create_download_link(res))